# **通过HuggingFace仓库中的Deepseek-R1模型进行微调，并使用训练后的模型验证**

# 1.安装依赖

In [1]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

# 2.登录huggingface

In [2]:
from huggingface_hub import login
from google.colab import userdata
hf_token=userdata.get('hugging-key')
login(hf_token)

# 3.初始化可视化功能



In [3]:
import wandb

wb_token = userdata.get("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B',
    job_type="training",
    anonymous="allow"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: getbytes (mustangai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


# 4.加载模型和token

In [4]:
from unsloth import FastLanguageModel
from google.colab import userdata
import torch
# 资源条件>=24GPU 可选择14/32B

max_seq_length = 2048
dtype = None
load_in_4bit = True
hf_token=userdata.get('hugging-key')
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
  )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.5: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.9k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

# 5.准备问题模板

In [5]:
prompt_style = """以下是描述任务的说明，并附有提供进一步背景的输入。
请写一个恰当的回应，完成该任务。
在回答之前，请仔细思考问题，并创建一个逐步的思维链，以确保逻辑清晰且准确。

### 任务描述:
您是一位在临床推理、诊断和治疗计划方面具有高级知识的医学专家。
请回答以下医学问题。

### 问题:
{}

### 回答:
<think>{}"""

# 6.微调前效果验证

In [6]:
question = "一位61岁的女性，有长期的在咳嗽或打喷嚏等活动中不自主漏尿的历史，但夜间没有漏尿。她接受了妇科检查和Q-tip试验。根据这些检查结果，膀胱容量测定（膀胱压力-容积曲线）最有可能显示她的残余尿量和膀胱逼尿肌的收缩情况是什么？"


FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")


from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

<｜begin▁of▁sentence｜>以下是描述任务的说明，并附有提供进一步背景的输入。
请写一个恰当的回应，完成该任务。
在回答之前，请仔细思考问题，并创建一个逐步的思维链，以确保逻辑清晰且准确。

### 任务描述:
您是一位在临床推理、诊断和治疗计划方面具有高级知识的医学专家。
请回答以下医学问题。

### 问题:
一位61岁的女性，有长期的在咳嗽或打喷嚏等活动中不自主漏尿的历史，但夜间没有漏尿。她接受了妇科检查和Q-tip试验。根据这些检查结果，膀胱容量测定（膀胱压力-容积曲线）最有可能显示她的残余尿量和膀胱逼尿肌的收缩情况是什么？

### 回答:
<think>
嗯，我现在需要解决一个关于膀胱容量测定的问题。患者是一位61岁的女性，有长期的在咳嗽或打喷嚏时不自主漏尿的历史，但夜间没有漏尿。她接受了妇科检查和Q-tip试验。现在需要根据这些检查结果，判断膀胱容量测定的残余尿量和膀胱逼尿肌收缩情况。

首先，我应该回忆一下膀胱容量测定（BCIS）的相关知识。BCIS通常包括测量膀胱压力和容积曲线。膀胱压力-容积曲线的形状通常显示出膀胱的容积与压力之间的关系。正常情况下，膀胱容积增加时，压力也会随之增加，但随着膀胱的充盈，压力会减小，这就是膀胱的压力-容积曲线。

患者有在咳嗽或打喷嚏时的不自主漏尿，这可能意味着膀胱在某些情况下无法完全排空，导致残余尿量增加。然而，夜间没有漏尿，这可能表明夜间膀胱的排空功能正常，或者夜间的排尿压力足够排出残余尿液。

Q-tip试验的结果通常用于评估尿道口周围的支持结构。如果Q-tip试验显示出高弯曲率，可能意味着尿道口下方的肌肉结构不支持膀胱，可能导致膀胱下垂或其他结构问题。

接下来，考虑膀胱容量测定的结果。膀胱容量测定显示出残余尿量和膀胱逼尿肌的收缩情况。残余尿量增加可能意味着膀胱没有完全排空，而膀胱逼尿肌的收缩情况可能与膀胱压力和容积曲线有关。

在这种情况下，患者可能有膀胱的低 tone，导致膀胱无法完全排空，尤其是在咳嗽或打喷嚏时。然而，夜间没有漏尿，可能表明夜间的膀胱排空压力足够高，能够排出残余尿液。

因此，结合这些信息，膀胱容量测定显示出患者的残余尿量增加，膀胱逼尿肌的收缩情况可能表现为低tone，导致膀胱在某些情况下无法完全排空，进而导致不自主漏尿。
</think>

根据膀胱容量测定的结果，患者的

# 7.准备微调数据集

In [7]:
train_prompt_style = """以下是描述任务的说明，并附有提供进一步背景的输入。
请写一个恰当的回应，完成该任务。
在回答之前，请仔细思考问题，并创建一个逐步的思维链，以确保逻辑清晰且准确。

### 任务描述:
您是一位在临床推理、诊断和治疗计划方面具有高级知识的医学专家。
请回答以下医学问题。

### 问题:
{}

### 回答:
<think>
{}
</think>
{}"""

In [8]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN
# 数据集格式化函数
def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [9]:
# 加载数据集
from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","zh", split = "train[0:200]")
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][0]

README.md:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

medical_o1_sft_Chinese.json:   0%|          | 0.00/64.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24772 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

'以下是描述任务的说明，并附有提供进一步背景的输入。\n请写一个恰当的回应，完成该任务。\n在回答之前，请仔细思考问题，并创建一个逐步的思维链，以确保逻辑清晰且准确。\n\n### 任务描述:\n您是一位在临床推理、诊断和治疗计划方面具有高级知识的医学专家。\n请回答以下医学问题。\n\n### 问题:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### 回答:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？

# 8.设置模型参数

In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.2.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# 9.设置训练（微调）参数

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

# 10.模型训练（微调)

In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 200 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,2.070600
20,1.623400
30,1.544300
40,1.485900
50,1.434700
60,1.403400


# 11.微调后验证

In [13]:
question = "一位61岁的女性，有长期的在咳嗽或打喷嚏等活动中不自主漏尿的历史，但夜间没有漏尿。她接受了妇科检查和Q-tip试验。根据这些检查结果，膀胱容量测定（膀胱压力-容积曲线）最有可能显示她的残余尿量和膀胱逼尿肌的收缩情况是什么？"


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)



<｜begin▁of▁sentence｜>以下是描述任务的说明，并附有提供进一步背景的输入。
请写一个恰当的回应，完成该任务。
在回答之前，请仔细思考问题，并创建一个逐步的思维链，以确保逻辑清晰且准确。

### 任务描述:
您是一位在临床推理、诊断和治疗计划方面具有高级知识的医学专家。
请回答以下医学问题。

### 问题:
一位61岁的女性，有长期的在咳嗽或打喷嚏等活动中不自主漏尿的历史，但夜间没有漏尿。她接受了妇科检查和Q-tip试验。根据这些检查结果，膀胱容量测定（膀胱压力-容积曲线）最有可能显示她的残余尿量和膀胱逼尿肌的收缩情况是什么？

### 回答:
<think>
这位61岁的女性有长期的咳嗽或打喷嚏时漏尿的历史，但在夜间没有漏尿，这让我想到她可能有某种类型的尿失禁。首先，我想到她可能有尿意不明确的尿失禁，也就是说她可能在某些时候无法控制尿液的释放。

接下来，提到了她接受了妇科检查和Q-tip试验，这让我想到这些检查可能帮助诊断尿失禁的具体原因。Q-tip试验通常用于检查尿道的位置和功能，尤其是当我们怀疑尿道下部的尿道括约肌功能不良时。

然后，膀胱容量测定听起来像是用来评估膀胱容量的实验。我想，膀胱容量测定通常用来评估膀胱的最大容量以及膀胱压力-容积曲线的形状，这有助于理解膀胱的舒张和收缩情况。

在考虑膀胱压力-容积曲线时，我想到了它的两个主要部分：膀胱压力和容积之间的关系。膀胱压力通常是指膀胱内的压力，而容积则是膀胱的容量。曲线可以帮助我们了解膀胱在不同容积下的压力情况。

现在，结合她有长期的漏尿历史，尤其是在咳嗽或打喷嚏时，这让我怀疑可能是膀胱的低压漏尿。这种情况下，膀胱在较低压力下就有可能漏尿，而在夜间没有漏尿，可能是因为夜间的膀胱压力较高，或者尿道括约肌在夜间更为强健。

再考虑到她接受的Q-tip试验结果，结合膀胱容量测定，我觉得这有助于进一步确定膀胱的功能状态。可能需要更多的信息来确认她的具体情况，但根据目前的信息，最有可能显示的是膀胱的低压漏尿情况。

最后，膀胱容量测定可以帮助我们更好地理解她的膀胱容量和膀胱逼尿肌的收缩情况。结合她的历史表现和检查结果，膀胱的低压漏尿是她最可能的尿失禁类型。
</think>
根据提供的信息，这位61岁的女性的尿失禁类型最有可能是膀胱的低压漏尿。膀胱容量测定（膀胱压力-容积曲线）能够显示她膀胱

# 12.保存微调后模型

In [14]:
# 保存到本地
new_model_local = "DeepSeek-R1-Medical-Doctor"
model.save_pretrained(new_model_local)
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.99 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 34%|███▍      | 11/32 [00:00<00:01, 12.95it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [01:06<00:00,  2.07s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving DeepSeek-R1-Medical-Doctor/pytorch_model-00001-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Medical-Doctor/pytorch_model-00002-of-00004.bin...


KeyboardInterrupt: 

In [ ]:
# 推送到huggingface仓库
new_model_online = "mustangai/DeepSeek-R1-Medical-Doctor"
model.push_to_hub(new_model_online)
tokenizer.push_to_hub(new_model_online)

model.push_to_hub_merged(new_model_online, tokenizer, save_method = "merged_16bit")